In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_1.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
...,...,...
17983,Losing a privilege to drive a car is a risk. S...,0.0
17984,There can be many advanteges and disadvanteges...,0.0
17985,What is the point of driving a car now and day...,0.0
17986,more people die in car related accidnts then a...,0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 4096])
Attention Mask size: torch.Size([17988, 4096])
Vocabulary Size: 29593
Max Sequence Length: 4096


In [7]:
model=Sequential()
input_length=4096
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=55000,output_dim=120,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)

Epoch 1/5
450/450 [==============================] - 942s 2s/step - loss: 0.0137 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 1823s 4s/step - loss: 2.9943e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 1404s 3s/step - loss: 1.5875e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 908s 2s/step - loss: 1.0002e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 981s 2s/step - loss: 6.8285e-06 - accuracy: 1.0000


In [11]:
xtest_np=xtest.numpy()

In [12]:
model.evaluate(xtest_np,ytest)


113/113 [==============================] - 48s 420ms/step - loss: 5.7089e-06 - accuracy: 1.0000


[5.708879598387284e-06, 1.0]

In [13]:
model.save("newmodel1.keras")

In [16]:
from keras.models import load_model
ml=load_model('newmodel1.keras')
xnew='Attending a four year high school has significant benefits that cannot be overlooked While some students may argue that attending a three year high school is just as beneficial there are several reasons why a four year high school is the better option Firstly attending a four year high school allows students to take more advanced courses Three year high schools may not offer the same level of advanced courses as four year high schools This means that students may not be able to take courses that are necessary for their future careers For example many colleges require students to take advanced math and science courses in high school and three year high schools may not offer these courses Secondly attending a four year high school allows students to participate in more extracurricular activities Many four year high schools offer a wider range of extracurricular activities than three year high schools This means that students can participate in activities that they are passionate about such as sports music or clubs These activities can also help students develop important skills such as teamwork leadership and communication Thirdly attending a four year high school can help students prepare for college Many colleges require students to take standardized tests such as the SAT or ACT in order to be considered for admission Four year high schools typically offer more resources and support to help students prepare for these tests This means that students who attend a four year high school may have a better chance of getting into their desired college Finally attending a four year high school can provide students with a more structured and supportive environment Four year high schools typically have larger student bodies than three year high schools which means that students can meet more people and make more friends Additionally four year high schools often have more resources such as counselors and tutors to help students with their academic and personal needs In conclusion attending a four year high school has significant benefits that cannot be overlooked From advanced courses to extracurricular activities college preparation and a supportive environment a four year high school can provide students with the tools they need to succeed in the future While some may argue that attending a three year high school is just as beneficial the evidence suggests that a four year high school is the better option'
xnew_tokens = tokenizer.encode(xnew,truncation=True,padding=True,return_tensors="np")
max_sequence_length = 4096
xnew_padded = pad_sequences(xnew_tokens,maxlen=max_sequence_length ,padding='post', truncating='post')
prediction = ml.predict(xnew_padded)
print(prediction)
# Set a threshold for classification
threshold = 0.5
prediction_value = prediction[0, 0]

# Check the prediction and print the result
if prediction_value >= threshold:
    print("1: It is an AI-generated text")
else:
    print("0: It is Human-generated text")

1/1 [==============================] - 0s 393ms/step
[[6.095884e-06]]
0: It is Human-generated text
